In [35]:
import pandas as pd
import numpy as np

def calculate_yield(yield_interest = 8, starting_fyt_interest = 1,
                    ending_fyt_interest = 12, step_size = .0001,
                    input_amount = 1000, liquidity = 50, gas_fee = .0019,
                    days_maturity = 90, days_matured = 0, target_interest = 0,
                    time_stretch = 15):
  columns = (
    'FYT Interest',
    'Actual Interest With Slippage',
    'Total Expenditure',
    'Received at Maturity',
    'APY'
  )

  values = []
  current_interest = ending_fyt_interest

  # zero out accumulated interest
  accumulated_interest = 0

  # Include time stretch
  t = (days_maturity - days_matured) / (365 * time_stretch)
  # target amount of underlying to accrue
  target_accrued = (target_interest * input_amount * (days_maturity - days_matured) / 365 / 100)

  for i in reversed(range (0, int(((ending_fyt_interest - starting_fyt_interest)/step_size)) + 1)):
    if days_matured > 0:
      accumulated_interest = input_amount * days_matured/365 * yield_interest/100

    # If the apy is 10% and there is 6 months remaining, the applied interest is 5%
    applied_interest = (days_maturity - days_matured) / 365 * current_interest

    y_start = liquidity
    x_start = y_start/((1-applied_interest/100)**(-1/t)-1)
    total_supply = x_start

    k = (y_start + total_supply)**(1-t) + x_start**(1-t)
    j = input_amount - accumulated_interest
    i = x_start - pow(k-pow(y_start + total_supply + j,1-t),1/(1-t))
    fee = (j - i) * .1
    i = i - fee

    fyt_discount_with_slippage = (1 - (i / j)) * 100

    fyt_expenditure = input_amount - accumulated_interest - i
    expenditure = fyt_expenditure + gas_fee + accumulated_interest
    received = (input_amount * yield_interest / 100) * (days_maturity) / 365
    apy = round(((received / expenditure) - 1) * 365 / (days_maturity - days_matured) * 100, 2)
    
    if ((received - expenditure) * 10) > target_accrued:
        row = []
        row.append(input_amount)
        row.append(current_interest)
        row.append(i/j)
        row.append(round(fyt_discount_with_slippage * (365/ (days_maturity - days_matured)), 5))
        row.append(expenditure)
        row.append(received)
        row.append(received - expenditure)
        row.append(apy)
        return row

    current_interest = current_interest - step_size


In [40]:
import ipywidgets as widgets

def button_run_on_click(_):
    status_label.value = "running...."

    columns = ['Input','FYT APY', 'FYT Price', 'Slippage APY', 'Spent', 'Received', 'Gain', 'APY']
    values = []

    for i in np.arange(.5, 5.1, .1):
      results = calculate_yield(yield_interest=yield_interest_input.value, \
                                input_amount = i, \
                                days_maturity = float(maturity_period_input.value), \
                                days_matured = days_matured_input.value, \
                                liquidity = liquidity_input.value, \
                                time_stretch = time_stretch_input.value, \
                                target_interest=target_interest_input.value)
      if results is not None:
        values.append(results)

    df = pd.DataFrame(values, columns = columns)
    pd.options.display.width = 20
    df.style.set_properties(**{'text-align': 'left'})
    
    status_label.value = ""

    result_box = setup_ui(df)

    main_box.children = [yield_interest_input, target_interest_input, maturity_period_input, days_matured_input, liquidity_input, time_stretch_input, button_run, status_label, result_box]


def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out
normal_label = widgets.Label()
normal_label.value = "Brute Force Calc"
status_label = widgets.Label()
status_label.layout.width = '300px'

layout = widgets.Layout(
    align_content='flex-start',
)
style = {
    'description_width':'initial'
}
yield_interest_input = widgets.FloatText(description='speculated interest:', value=8, style=style, layout=layout)
target_interest_input = widgets.FloatText(description='target interest:', value=16, style=style, layout=layout)
maturity_period_input = widgets.Dropdown(description='tranche length:', options=['90', '120', '180'],style=style, layout=layout)
days_matured_input = widgets.FloatText(description='days matured:', value=0, style=style, layout=layout,disabled=True)
liquidity_input = widgets.FloatText(description='liquidity:', value=50, style=style, layout=layout)
time_stretch_input = widgets.FloatText(description='time stretch:', value=15, style=style, layout=layout)
button_run = widgets.Button(description="Run")
main_box = widgets.VBox([normal_label,yield_interest_input, target_interest_input, maturity_period_input, days_matured_input, liquidity_input, time_stretch_input, button_run, status_label])
button_run.on_click(button_run_on_click)
display(main_box)


In [19]:
def jonny(speculated_interest=8,input_amount=.5,days_maturity=90,\
          liquidity=50,time_stretch=15,gas_fee=0.0019,\
          target_interest=16):
    t = (days_maturity) / (365 * time_stretch)
    T = (days_maturity) / 365
    y_start = liquidity

    A = (input_amount*(speculated_interest/100)*T-(target_interest/1000)*input_amount*T-gas_fee)/input_amount
    current_interest = A/T*100
    x_start=y_start/((1-T*(current_interest/100))**(-1/t)-1)
    unit_price = ((y_start+x_start)/x_start)**-t
    expenditure=input_amount-unit_price*input_amount+gas_fee
    received = (input_amount * speculated_interest / 100) * (days_maturity) / 365
    apy = round(((received / expenditure) - 1) * 365 / (days_maturity) * 100, 2)
    row = []
    row.append(input_amount)
    row.append("?")
    row.append(unit_price)
    row.append(A/T*100)
    row.append(expenditure)
    row.append(received)
    row.append(received - expenditure)
    row.append(apy)
    return row

In [41]:
import ipywidgets as widgets

def button_run_on_click_j(_):
    status_label_j.value = "running...."

    columns = ['Input','FYT APY', 'FYT Price', 'Slippage APY', 'Spent', 'Received', 'Gain', 'APY']
    values = []

    for i in np.arange(.5, 5.1, .1):
      results = jonny(speculated_interest=yield_interest_input_j.value, \
                                input_amount = i, \
                                days_maturity = float(maturity_period_input_j.value), \
                                liquidity = liquidity_input_j.value, \
                                time_stretch = time_stretch_input_j.value, \
                                target_interest=target_interest_input_j.value)
      if results is not None:
        values.append(results)

    df = pd.DataFrame(values, columns = columns)
    pd.options.display.width = 20
    df.style.set_properties(**{'text-align': 'left'})
    
    status_label_j.value = ""

    result_box_j = setup_ui(df)

    main_box_j.children = [yield_interest_input_j, target_interest_input_j, maturity_period_input_j, days_matured_input_j, liquidity_input_j, time_stretch_input_j, button_run_j, status_label_j, result_box_j]


def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out
jonny_label = widgets.Label()
jonny_label.value = "Instant Calc"
status_label_j = widgets.Label()
status_label_j.layout.width = '300px'

layout = widgets.Layout(
    align_content='flex-start',
)
style = {
    'description_width':'initial'
}
yield_interest_input_j = widgets.FloatText(description='speculated interest:', value=8, style=style, layout=layout)
target_interest_input_j = widgets.FloatText(description='target interest:', value=16, style=style, layout=layout)
maturity_period_input_j = widgets.Dropdown(description='tranche length:', options=['90', '120', '180'],style=style, layout=layout)
days_matured_input_j = widgets.FloatText(description='days matured:', value=0, style=style, layout=layout,disabled=True)
liquidity_input_j = widgets.FloatText(description='liquidity:', value=50, style=style, layout=layout)
time_stretch_input_j = widgets.FloatText(description='time stretch:', value=15, style=style, layout=layout)
button_run_j = widgets.Button(description="Run")
main_box_j = widgets.VBox([jonny_label,yield_interest_input_j, target_interest_input_j, maturity_period_input_j, days_matured_input_j, liquidity_input_j, time_stretch_input_j, button_run_j, status_label_j])
button_run_j.on_click(button_run_on_click_j)
display(main_box_j)